# Die Casting Automatic Defect Detection

## Project Description
Casting is a manufacturing process in which a liquid material is poured into a mold that contains a hollow cavity of
the desired shape and then allowed to solidify. There are three types of casting processes: Sand Casting, Die Casting,
and Lost Wax Casting. There are many types of defects in casting like holes, burr, shrinkage defects, mold material
defects, pouring metal defects, metallurgical defects, etc. A foundry that produces bearing bushes, in order of 10,000
parts per day has invested large funding to automate the process of finding defects in its casting production line.
Currently, the inspection process is carried out manually by QC personnel. It is a very time-consuming process and due
to human error, the process of rejecting defects is not very accurate. This can be the cause of rejection of an entire
order which would lead to huge losses. The goal is to build a Machine Learning Model to eliminate this loss of revenue
and make the QC process as accurate as possible.

Dataset: The zip folder contains the photos of the parts. All images are (512x512) pixels grey-scaled. There are two
Folders, the Training, and the Test set, each contains two subfolders of “Defect” and “Okay” parts.  

Your Task, develop a deep learning classification model for this problem. Evaluate and validate your model accuracy
with this Test set. Later, try to use the model to predict if a single unknown image (some examples in the Folder,
New_set) is a defective or okay part.<br>

Source of the dataset: https://www.kaggle.com/

## Importing the Libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

2021-11-16 15:41:12.500718: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-16 15:41:12.500790: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Data Preprocessing

### Preprocessing the Training Set

In [2]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
training_set = train_datagen.flow_from_directory('../../data/Casting/Training_Set',
                                                 target_size=(64, 64),
                                                 batch_size=32,
                                                 class_mode='binary')

Found 7383 images belonging to 2 classes.


### Preprocessing the Test Set

In [3]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_set = test_datagen.flow_from_directory('../../data/Casting/Test_Set',
                                            target_size=(64, 64),
                                            batch_size=32,
                                            class_mode='binary')

Found 863 images belonging to 2 classes.


## Building the CNN Model

### Step 1 - Initialising the Model

In [4]:
Model = tf.keras.models.Sequential()

2021-11-16 15:41:14.758844: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-16 15:41:14.758866: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-16 15:41:14.758885: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (liamb-surface): /proc/driver/nvidia/version does not exist
2021-11-16 15:41:14.759116: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Step 2 - Adding First Convolution Layer 

In [5]:
Model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 3 - Pooling the First Layer

In [6]:
Model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=1))

### Step 4 - Adding a Second Convolutional Layer

In [7]:
Model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

### Step 5 - Pooling the Second Layer

In [8]:
Model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=1))

### Step 6 - Flattening

In [9]:
Model.add(tf.keras.layers.Flatten())

### Step 7 - Full Connection

In [10]:
Model.add(tf.keras.layers.Dense(units=256, activation='relu'))

2021-11-16 15:41:14.880960: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 110231552 exceeds 10% of free system memory.
2021-11-16 15:41:14.965548: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 110231552 exceeds 10% of free system memory.
2021-11-16 15:41:14.998150: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 110231552 exceeds 10% of free system memory.


### Step 8 - Output Layer

In [11]:
Model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Step 9 - Compiling the CNN

In [12]:
Model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Training the CNN and Evaluation

In [13]:
Model.fit(x=training_set, validation_data=test_set, epochs=50)

2021-11-16 15:41:15.581678: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-16 15:41:15.751487: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 110231552 exceeds 10% of free system memory.
2021-11-16 15:41:15.779802: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 110231552 exceeds 10% of free system memory.


Epoch 1/50
231/231 [==============================] - 93s 401ms/step - loss: 0.8000 - accuracy: 0.6113 - val_loss: 0.6820 - val_accuracy: 0.5782

KeyboardInterrupt: 

## Making a Prediction

In [ ]:
import numpy as np
from keras.preprocessing import image

test_image = image.load_img('../../data/Casting/InLineProduction_Detection/Part10.jpeg', target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

result = Model.predict(test_image)

print(training_set.class_indices)

if result[0][0] == 1:
    prediction = 'Part is Okay.'
else:
    prediction = 'Part is Defective!'
print(prediction)